# File Change Semantics

#### Anmol Eswarapu

Heim introduced the concept of file change semantics, a revival and modification of an old approach to thinking about indeterminates and determinates in multi-sentence discourses. In this project, I will be attempting to implement a file change semantic parser to parse a few simple sentences. I will explain the strengths and shortcomings of my alogrithm and discuss where it can be improved.

This project will be divided into creating a context free grammar that can generate the sentences I wish to explore (but also overgenerates), a generator which will generate grammatically correct strings and syntax trees given our CFG, and our semantic parser.

## Contents

1. [What is File Change Semantics?](#What-is-File-Change-Semantics?)
2. [CFG and Generator](#CFG-and-Generator)
    1. [CFG](#CFG)
    2. [Generator](#Generator)
3. [Syntax Tree Functions](#Syntax-Tree-Functions)
4. [Exploring Generated Strings](#Exploring-Generated-Strings)
5. [Semantic Parser](#Semantic-Parser)
    1. [Initializing](#Initializing)
    2. [Updating](#Updating)
    3. [Definite Articles](#Definite-Articles)
    4. [Final Algorithm](#Final-Algorithm)
6. [Testing](#Testing)
7. [Summary and Thoughts](#Summary-and-Thoughts)

## What is File Change Semantics?

Heim's File Change Semantics is an approach to understanding the difference between indefinite articles and definite articles. Heim argues that when a new object is introduced into a discourse, it should be introduced with an indefinite. On the other hand, an object is paired with a definite only when that object is already in the discourse - either it has already been introduced in a prior phrase, or it is part of the common knowledge. (For example, `a woman looked at the sun` is perfectly meaningful even though we never introduced `the sun` in this discourse because we already know what `the sun` is.)

Consider $(i)$:



    a) A woman was comforted by a dog. b) The woman pet the dog. c) The dog jumped over a fence.
    
$ \tag i $

According to Heim, we want to have a file, call it $F_0$, that starts off empty. 

|$F_0$|
|-|

After a) is uttered, two cards are put into the file, each labelled with a number, call them $0$ and $1$. We want the file cards to look as follows:


|$F_1$|$0$| $1$|
|-|--| --|
||- woman|- dog|
||- was comforted by $1$|- comforted $0$|

Each card is supposed to represent a discourse referrent (for more information, read Heim's 1983 paper).

So $F_0$ is updated, now call it $F_1$

After b) is uttered, $0$ and $1$ are updated; no new cards are added.

|$F_2$|$0$| $1$|
|-|-| --|
||- woman|- dog|
    ||- was comforted by $1$|- comforted $0$|
||- pet $1$|- was pet by $0$|

Finally, after c) is uttered, a new card is added, call it $2$. Card $1$ is also updated as follows:

|$F_3$|$0$| $1$|$2$
|-|-| --|-|
||- woman|- dog|- fence|
||- was licked by $1$|- comforted $0$|- $1$ jumped over|
||- pet $1$|- was pet by $0$||
|||- jumped over $1$||

This file represents all the semantic knowledge contained in $(i)$. The basic algorithm is straightfoward and the knowldge base can easily be updated, both when it comes to new objects as well as new details about previous objects.

My task is to implement Heim's procedure in python.

## CFG and Generator

### CFG

I modify a few sentences from Heim (1983):

'A woman was comforted by a dog. The woman pet the dog. The dog jumped over a fence.' $\tag i$



I've modified these sentences from their originals to remove pronouns (`it` $\rightarrow$ `the dog`, and `she` $\rightarrow$ `the woman` ) and also to make the subject matter less violent. I remove pronouns because the focus of this project is anaphoric determinates, and pronouns add a layer of complexity that is not needed for our focus. I make the subject matter less violent because I generally don't like violence. 

A CFG for this discourse is as follows:

In [1]:
cfg = {
    'S'     : [('NP', 'VP')],                                             ## S  --> NP VP
    'NP'    : [('INDEF', 'N'), ('DEF', 'N')],                             ## NP --> INDEF N | DEF N           
    'VP'    : [('V', 'NP'), ('V', 'PP'), ('Vaux', 'Vbar')],               ## VP --> V NP    | Vaux VPP
    'Vbar'  : [('V', 'PP')],                                              ## etc.
    'PP'    : [('PREP', 'NP')],
    
    'INDEF' : [('a',)],
    'DEF'   : [('the',)],
    'N'     : [('woman',), ('dog',), ('fence',)],
    'Vaux'  : [('was',)],
    'V'     : [('comforted',), ('pet',), ('jumped',)],
    'PREP'  : [('by',), ('over',)],
}

### Generator

This is a modification of the CFG generator that we created in class. Rather than outputting a string, this function creates a syntax tree, which allows for easier analysis later on.

In [2]:
import random

In [3]:
## syntax tree class

class Tree(object):
    def __init__(self, object):
        self.left = None
        self.right = None
        self.data = object
        self.head = None
        
## initialize root of tree

test = Tree('S')

In [4]:
## generator function

def gen(gram, ptr):
    
    
    branches = random.choice(gram[ptr.data])
    
    #print(branches[1], 'branches')
                
    for val, branch in enumerate(branches): ##for each branch of RHS,
        if val == 0:
            ptr.left = Tree(branch) ## add to tree
            ptr.left.head = ptr
            
        if val == 1:
            ptr.right = Tree(branch)
            ptr.right.head = ptr
            
        if branch in gram: ## if tree can be continued
            if val == 0:
                gen(gram, ptr.left)
            else:
                gen(gram, ptr.right)
                
    return ptr

In [5]:
gen(cfg, test)

## Syntax Tree Functions

Now that we have a syntax tree, we want functions to read and traverse this tree:

In [6]:
## print end nodes

def LeafNodes(ptr):
    res = []
    if ptr == None: ## if node has no data
        return
    
    if ptr.left == None and ptr.right == None: ## if node has no children
        res.append(ptr.data)
        return res
    
    if ptr.left != None:
        res = res + LeafNodes(ptr.left)
    if ptr.right != None:
        res = res + LeafNodes(ptr.right)
    
    return res

In [7]:
LeafNodes(test)

['the', 'fence', 'was', 'jumped', 'over', 'the', 'fence']

In [8]:
## return a list of subtrees specified by POS

def findNode(ptr, pos):
    res = []
    if ptr != None:
        if ptr.data is pos:
            res.append(ptr)
        res = res + findNode(ptr.left, pos)
        res = res + findNode(ptr.right, pos)
        
    return res

In [9]:
for x in findNode(test, 'NP'):
    print(LeafNodes(x))
    
print('')
for x in  findNode(test, 'VP'):
    print(LeafNodes(x))

['the', 'fence']
['the', 'fence']

['was', 'jumped', 'over', 'the', 'fence']


## Exploring Generated Strings

According to file change semantics, the first time a new object is introduced to the discourse, it must be accompanied with an indeterminate article. As can be seen by some of the sample sentences below, this is not always the case with our CFG. We can say that this grammar overgenerates: though it does generate all the strings we are interested in, e.g. $(i)$ above, it is also able to generate strings that we deem ungrammatical for our purposes. Nonetheless, the CFG is sufficient; we just have to be selective of which sentences we choose when testing our program. 


It's also interesting to note that while some of these sentences might seem awkward at first glance, they are actually all gramatically correct in English. For instance, below we have the sentence `the fence comforted by a fence`. While this is unwieldly, it can make sense: the fence comforted something, and it did so by a fence. All other sentences generated with this CFG should be similarly sensible.

In [10]:
random.seed( 1502 )

for x in range(10):
    gen(cfg, test)
    print(' '.join(LeafNodes(test))+'.\n')

the dog was jumped over a dog.

the fence comforted over a dog.

a woman was jumped over the woman.

a woman comforted the dog.

a fence was pet by a fence.

the woman pet over a fence.

the fence jumped over a woman.

the woman was pet over the dog.

a woman jumped by the woman.

a fence jumped by a woman.



## Semantic Parser

### Initializing

The first step in  the FCS procedure is identifying any indefinite Noun Phrases and adding the corresponding Noun to the file. Like Heim, I will label each card with a number. I'll be using Python's dictionary data structure to represent the file, with the key/value pairs being card label/card contents.

In [11]:
## print a test sentence
LeafNodes(test)

['a', 'fence', 'jumped', 'by', 'a', 'woman']

In [12]:
def initialize(ptr, file):
    m = len(file)
    
    n = len(findNode(ptr, 'INDEF')) ## number of INDEF NP
    for i, val in enumerate(findNode(ptr, 'INDEF')):
        x = val.head.right.left.data ##associated noun
        if x not in file:
            file[m+i] = [x] ##add card to file
    return file

In [13]:
file1 = {}
file1 = initialize(test, file1)

for x in file1:
    print( x, '\n', file1[x], '\n')

0 
 ['fence'] 

1 
 ['woman'] 



### Updating

The next step in the procedure is identifying any descriptive phrases associated with each noun, and updating the file cards appropriately. Since the verbs in our model are all transitive, there are two ways a card can be updated: the noun in question can be either the subject or the object of the sentence, and so the update algorithm has to be appropriately specified.

In [14]:
def updateObject(ptr, file, noun):
    desc = LeafNodes(ptr)[:-2] ## description is S - last NP
    
    for x in file:
        for y in range(len(desc)-1):
            if file[x][0] == desc[y]:
                desc[y] = str(x) ## replace noun with card label
                desc.pop(y-1)
                
    for x in file:
        if file[x][0] == noun:
            file[x].append(' '.join(desc))
    return file

In [15]:
def updateSubject(ptr, file, noun):
    desc = LeafNodes(ptr)[2:] ## description is S - first NP
    
    for x in file:
        for y in range(len(desc)):
            if file[x][0] == desc[y]:
                desc[y] = str(x)
                desc.pop(y-1)
                
    for x in file:
        if file[x][0] == noun:
            file[x].append(' '.join(desc))
    return file

In [16]:
def update(ptr, file):
    for x in file:
        noun = file[x][0]
        if findNode(ptr, noun):
            
            if findNode(ptr, noun)[0].head.head.head.data == 'S':
                updateSubject(ptr, file, noun)
            else:
                updateObject(ptr, file, noun)

In [17]:
## recall test sentence
LeafNodes(test)

['a', 'fence', 'jumped', 'by', 'a', 'woman']

In [18]:
update(test, file1)
for x in file1:
    print(x, '\n', file1[x], '\n')

0 
 ['fence', 'jumped by 1'] 

1 
 ['woman', '0 jumped by'] 



### Definite Articles

The next part of the process is dealing with definite articles. If a noun phrase has a definite article, then the noun w as already introduced. Thus, no card has to be initialized, but updating is identical from before alebit with a new sentence.

In [19]:
LeafNodes(test)

['a', 'fence', 'jumped', 'by', 'a', 'woman']

In [20]:
random.seed(217)

test2 = Tree('S')
gen(cfg, test2)
LeafNodes(test2)

['a', 'dog', 'was', 'pet', 'over', 'the', 'fence']

In [21]:
initialize(test2, file1)
update(test2, file1)

In [22]:
for x in file1:
    print(x, '\n', file1[x], '\n')

0 
 ['fence', 'jumped by 1', '2 was pet over'] 

1 
 ['woman', '0 jumped by'] 

2 
 ['dog', 'was pet over 0'] 



### Final Algorithm

Putting together our various finctions, we end up with a small, tody program capable of file change semantic parsing.

In [23]:
def filechangesemantics(discourse, file):
    
    # discourse : a list of sentence tree roots
    # file : a dict to store card info (key/val paries of card label/card info)
    
    for sentence in discourse:
        initialize(sentence, file)
        update(sentence, file)
    return file

In [24]:
def prettyprint(file):
    # prints filecards in a pretty way
    
    for x in file:
        print(x, '\n', file[x], '\n')

## Testing

Generating three grammatical sentences, we'll evaluate what our algorithm outputs:

In [25]:
root1, root2, root3 = Tree('S'), Tree('S'), Tree('S')

In [26]:
random.seed(6410)

gen(cfg, root1)
LeafNodes(root1)

['a', 'fence', 'pet', 'a', 'dog']

In [27]:
random.seed(7959)

gen(cfg, root2)
LeafNodes(root2)

['the', 'dog', 'pet', 'by', 'a', 'woman']

In [28]:
random.seed(1299)

gen(cfg, root3)
LeafNodes(root3)

['the', 'woman', 'pet', 'over', 'the', 'fence']

In [29]:
discourse1 = [root1, root2, root3]

file = {}
file = filechangesemantics(discourse1, file)
prettyprint(file)

0 
 ['fence', 'pet 1', '2 pet over'] 

1 
 ['dog', '0 pet', 'pet by 2'] 

2 
 ['woman', '1 pet by', 'pet over 0'] 



## Summary and Thoughts

### Efficiency

The algorithm runs in `O(nm)` time at worst, and `O(n)` at best, where `n` =  total number of words in all sentences, and `m` = number of determinate `NP`s. Each time a determinate `NP` is found, the parser has to look through its file to see if that card already exists. This is an incredibly efficient algorithm, which makes it usable for both real world applications, and a candidate for explaining what's going on in our heads.

### Edge Cases
While its great to see the algorithm work on proper inputs, it's interesting to see what the algorithm does with improper inputs, e.g. objects paired with determinates but not yet properly introduced in the discourse:

In [30]:
random.seed(198)

root = Tree('S')
gen(cfg, root)

LeafNodes(root)

['the', 'fence', 'was', 'comforted', 'by', 'the', 'dog']

In [31]:
prettyprint(filechangesemantics([root], {}))

The function outpupts nothing, which is exactly what we want; since we have no card already in place for fence or dog, there is no card to update. and so we see no files being initialized or updated.

In [32]:
random.seed(773)

root = Tree('S')

gen(cfg, root)
LeafNodes(root)

['the', 'fence', 'jumped', 'over', 'a', 'dog']

In [33]:
prettyprint(filechangesemantics([root], {}))

0 
 ['dog', 'the fence jumped over'] 



This time, the dog was able to be initialized, while the fence wasn't. Again, exactly what we would expect.

What's interesting about these failures is that we can see that our semantic parser is actually able to differentiate between indefinites and definites; the parser does not look at the syntax, but purely the article type when it chooses whether to initialize another object. Some of the grammaticality resides in the semantics of the discourse, rather than the syntax of the sentences. After all, `The fence jumped over a dog` would be considered ungrammatical as the first sentence in a discourse, but might be grammatical if its the second or third.

As mentioned before, sentences like `A woman looked at the sun` seem to be problematic. However, "common knowledge" objects like `the sun` are actually easily accounted for; rather than starting with an entirely empty file, we simply need a starting file with cards already in place for common knowledge objects - `the sun`, `the president`, etc. Even sentences like `A man ran to the store`, where `the store` is actually an arbitrary store, rather than a particular one, can make sense with a sophisticated enough starting file.

In [34]:
random.seed(55)

root = Tree('S')
gen(cfg, root)
LeafNodes(root)

['a', 'fence', 'comforted', 'over', 'a', 'fence']

In [35]:
random.seed(299)

root2 = Tree('S')
gen(cfg, root2)
LeafNodes(root2)

['a', 'dog', 'was', 'jumped', 'by', 'the', 'fence']

In [36]:
prettyprint(filechangesemantics([root, root2], {}))

0 
 ['fence', 'comforted over 0', 'comforted over 0', '2 was jumped by', '2 was jumped by'] 

1 
 ['fence', 'comforted over 0', 'comforted over 0', '2 was jumped by', '2 was jumped by'] 

2 
 ['dog', 'was jumped by 0'] 



Sentences like `a fence comforted over a fence` are problematic, and our algorithm cannot deal with it properly, as can be seen here. Since `fence` is introduced to the discourse twice, two `fence` cards are initialized. However, there's no way to tell which card should be updated when new information is introduced. Nor can we tell which fence should be mentioned in other cards. But this makes sense, and is analogous to how we, as humans, would deal with such ambiguity in normal conversation. If the sentences above were uttered to us, we would only be confused and unable to properly process the semantic information. We might very well also create two `fence` cards, and update both of them when new information about either one is brought up. Or we might do nothing at all. While our algorithm doesn't deal with it the best way possible, it still does an admirable job.

## Further Research

Singular Pronouns are somewhat easy to implement; treating them as determinate `NP`s works to an extent, though dealing with the anaphora is troublesome. If the parser ever sees a `she`, for example, we would have to tell it to update a card belonging to a `woman`, or a `girl`, or a `mom`, etc. However, the ambiguity that might arise from such a procedure would be hard to deal with in many cases.


While the parser itself is sophisticated enough, our CFG isn't. Since some of the grammaticality lies in the semantics, rather than purely in the syntax, this isn't surprising. Nonetheless, we could've created multiple CFGs, one for the first sentence in a discourse, and one for other sentences. Doing so would have made it easier to create proper discourses to test our parser.

This program on the whole is a successful implementation. It can parse the first few sentences that Heim offers in her paper. However, a lot of work can still be done. Other than the suggestions I've already offered here, dealing with more complex grammar, such as relative clauses or even just adjective phrases, would be good places to continue researching.